# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

It is classification problem as we need to seperate data into two labels one being students who needs intervention and students who dont need intervention.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [2]:
# Import libraries
import numpy as np
import pandas as pd


In [3]:
# Read student data
global student_data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
#print (student_data['passed'])
#Randomizing student data
#student_data = student_data .iloc[np.random.permutation(len(student_data))]

# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [4]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
#print (student_data.shape)
n_students = student_data.shape[0]
#print(n_students)
n_features = student_data.shape[1]-1
#print(n_features)
passed = (student_data['passed'] == 'yes')
#print type(passed)
n_passed = student_data[passed].shape[0]
#print(n_passed)

failed = (student_data['passed'] == 'no')
n_failed = student_data[failed].shape[0]
grad_rate = ((float(n_passed) / float(n_students))*100)

print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)



Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [5]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
print (type(feature_cols))
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

<type 'list'>
Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3 

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [6]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty
    #print(outX)
    # Check each column
    for col, col_data in X.iteritems():
        #print col
        #print col_data
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'
        #print (col_data)
        outX = outX.join(col_data)  # collect column(s) in output dataframe
        #print (outX)
    return outX

##global X_all
X_all = preprocess_features(X_all)
#print (X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))
print (X_all).head

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
<bound method DataFrame.head of      school_GP  school_MS  sex_F  sex_M  age  address_R  address_U  \
0            1          0      1      0   18          0          1   
1            1          0      1      0   17          0          1   
2            1          0      1      0   15          0          1   
3            1          0 

### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [39]:
# First, decide how many training vs test samples you want
#num_all = student_data.shape[0]  # same as len(student_data)
#num_train = 300  # about 75% of the data
#num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.24, random_state=42)

#X_train = X_all[:num_train]
#X_test = X_all[num_train:num_all]
#y_train = y_all[:num_train]
#y_test = y_all[num_train:num_all]
#print (X_train.head)
#print (y_train.head)
#print (X_test.head)
#print (y_test.head)


print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

#X_train.to_csv('out.csv')

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [40]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
#import ?

from sklearn.naive_bayes import GaussianNB
clf =  GaussianNB()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training GaussianNB...
Done!
Training time (secs): 0.002


In [41]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.80378250591


In [42]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.763358778626


In [61]:
# Train and predict using different training set sizes
from sklearn.naive_bayes import GaussianNB
clf =  GaussianNB()
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
training_size = [ 100, 200, 300]
for x in training_size:
    X_train = X_all[:x]
    print (X_train.shape)
    y_train = y_all[:x]
    print (y_train.shape)
    train_predict(clf, X_train, y_train, X_test, y_test)

(100, 48)
(100L,)
------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.014
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.453608247423
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.282051282051
(200, 48)
(200L,)
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.002
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.807142857143
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.75
(300, 48)
(300L,)
------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.003
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.824644549763
Predicting labe

In [62]:
from sklearn import tree
from sklearn.externals.six import StringIO 
from pandas import DataFrame
from os import system
from IPython.display import Image  
#import pydot2
clf = tree.DecisionTreeClassifier(random_state=40)
training_size = [ 100, 200, 300]
for x in training_size:
    X_train = X_all[:x]
    #print (X_train.shape)
    y_train = y_all[:x]
    #print (y_train.shape)
    #exporting tree to a dot format so that we can import it from graph vizualizer to display the tree structure
    train_predict(clf, X_train, y_train, X_test, y_test)

#Priniting important features in dataframe
#print DataFrame(clf.feature_importances_, columns = ["Imp"], index = X_train.columns).sort(['Imp'], ascending = False)

#exporting training data to a csv file
#X_train.to_csv('out1.csv')
#y_train.to_csv('out2.csv')

#exporting tree to a dot format so that we can import it from graph vizualizer to display the tree structure
#tree.export_graphviz(clf, out_file='C:/Users/jac812i/dtree6.dot', feature_names = X_train.columns)






------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.796875
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.766666666667
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Pr

In [59]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=40)
training_size = [ 100, 200, 300]
for x in training_size:
    X_train = X_all[:x]
    print (X_train.shape)
    y_train = y_all[:x]
    print (y_train.shape)
    train_predict(clf, X_train, y_train, X_test, y_test)



    

(100, 48)
(100L,)
------------------------------------------
Training set size: 100
Training RandomForestClassifier...
Done!
Training time (secs): 0.056
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.006
F1 score for training set: 0.993288590604
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.776119402985
(200, 48)
(200L,)
------------------------------------------
Training set size: 200
Training RandomForestClassifier...
Done!
Training time (secs): 0.037
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.992647058824
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.881355932203
(300, 48)
(300L,)
------------------------------------------
Training set size: 300
Training RandomForestClassifier...
Done!
Training time (secs): 0.055
Predicting labels using RandomForestC

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [69]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn import tree
model = tree.DecisionTreeClassifier(random_state=40)

f1_scorer = make_scorer(f1_score, pos_label="yes")

parameters = { 
        'criterion': ["gini","entropy"],
        'max_depth': [5,10,15,20,25],
   # 'min_samples_split': [5, 10, 15]
    #'min_samples_leaf': [5, 10, 15]
}

#clf = RandomForestClassifier(n_estimators= 500,max_features = 'auto', random_state=40)
clf = GridSearchCV(model, parameters, n_jobs=1,scoring=f1_scorer)
clf.fit(X_train, y_train)
best_estimator = clf.best_estimator_

print "Best fit parameter = " + str(clf.best_params_)
print "Best score = " + str(clf.best_score_)
print 'Best parameters set:'
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print '\t%s: %r' % (param_name, best_parameters[param_name])


# Returns the best configuration for a model using crosvalidation
# and grid search

#clf.fit(X_train, y_train)

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for train set: {}".format(train_f1_score) 
    
test_f1_score = predict_labels(clf, X_test, y_test)
print "F1 score for final test set: {}".format(test_f1_score) 
   


Best fit parameter = {'criterion': 'gini', 'max_depth': 15}
Best score = 0.726481821686
Best parameters set:
	criterion: 'gini'
	max_depth: 15
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.000
F1 score for train set: 1.0
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.000
F1 score for final test set: 0.888888888889
